# Import Dataset

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

width = 64
height = 64

In [2]:
df = pd.read_feather('velocity_fields_all_times.feather')


In [3]:
df_idx = df.set_index(['scene','t','x','y'])
del(df)
df_xr = df_idx.to_xarray()
del(df_idx)
np_data = df_xr.to_dataarray().to_numpy()
del(df_xr)
np_data = np.moveaxis(np_data, 0, -1)
print(np_data.shape)

(1000, 6, 64, 64, 3)


# Generate Input Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import seaborn as sns

In [ ]:
width = 35
height = 35
n_examples = 100
max_sigma = 3.0
max_val = 1
rng = np.random.default_rng()

In [ ]:
def heatmap(raw_data):
  magnitude = np.linalg.norm(raw_data, axis=2)
  sns.heatmap(magnitude)
  plt.show()
  return

In [ ]:
from matplotlib.animation import FuncAnimation

def quiver(raw_data, ax=None):
  if ax is None:
    ax = plt.gca()
  X, Y = np.meshgrid(np.linspace(0,1,width), np.linspace(0,1,height))
  U = raw_data[:,:,0]
  V = raw_data[:,:,1]
  magnitude = np.linalg.norm(raw_data, axis=2)
  return ax.quiver(X, Y, U, V, magnitude)

fig, ax = plt.subplots(figsize=(8, 6))

data = np_data[0, 0, :, :, :2]
X, Y = np.meshgrid(np.linspace(0,1,width), np.linspace(0,1,height))
U = data[:,:,0]
V = data[:,:,1]
magnitude = np.linalg.norm(data, axis=2)
quiver_plot = ax.quiver(X, Y, U, V, magnitude)
colorbar = plt.colorbar(quiver_plot, ax=ax)

def update(frame):
    ax.clear()
    data = np_data[0, frame, :, :, :2]
    X, Y = np.meshgrid(np.linspace(0,1,width), np.linspace(0,1,height))
    U = data[:,:,0]
    V = data[:,:,1]
    magnitude = np.linalg.norm(data, axis=2)
    quiver_plot = ax.quiver(X, Y, U, V, magnitude)
    ax.set_title(f'Frame {frame}')
    return quiver_plot,

anim = FuncAnimation(fig, update, frames=11, interval=200, blit=False)
anim.save('velocity_animation.gif', writer='pillow', fps=5)
plt.show()

In [ ]:
def generate_frame(sigma=2):
  data = rng.random((width, height, 2))
  data = (data * max_val * 2) - max_val
  blur = np.random.uniform(low=0.5, high=max_sigma)
  data = gaussian_filter(data, sigma=(blur, blur, 0))
  return data

In [ ]:
def generate_inputs(n):
  return np.array([generate_frame() for _ in range(n)])

In [ ]:
inputs = generate_inputs(n_examples)
for i in range(5):
  frame = inputs[i]
  quiver(frame)

# Generate Labels

In [ ]:
def get_kinetic_engergy(frame):
  kinetic_engergy = np.sum((0.5 * np.sum(np.square(frame), axis=2)))
  return kinetic_engergy

In [ ]:
def get_reynolds_number(frame):
    velocity_magnitude = np.linalg.norm(frame, axis=2)
    mean_velocity = np.mean(velocity_magnitude)
    characteristic_length = min(width, height)
    kinematic_viscosity = 1.0
    reynolds_number = (mean_velocity * characteristic_length) / kinematic_viscosity
    return reynolds_number

In [ ]:
for i in range(5):
  print(get_reynolds_number(np_data[i]))
  quiver(np_data[i])

In [ ]:
outputs = np.array([get_kinetic_engergy(input) for input in inputs])

# Generate graph representation

In [ ]:
import torch_geometric as pyg
from torch_geometric.loader import DataLoader
import networkx as nx
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device {device}')

In [ ]:
def coords_to_id(x_pos, y_pos):
    return width * x_pos + y_pos
def id_to_coords(id):
    return (id // width, id - width * (id // width))

In [ ]:
n_examples = 1000
inputs = generate_inputs(n_examples)

Construct an edge list for a regular 2d grid representation of the fluid

In [ ]:
edge_list = []
for x_pos in range(inputs.shape[1]):
    for y_pos in range(inputs.shape[2]):
        if(x_pos != inputs.shape[1]-1):
            edge_list.append([coords_to_id(x_pos, y_pos), coords_to_id(x_pos+1, y_pos)])
        if(y_pos != inputs.shape[2]-1):
            edge_list.append([coords_to_id(x_pos, y_pos), coords_to_id(x_pos, y_pos+1)])
        if((x_pos != inputs.shape[1]-1) and (y_pos != inputs.shape[2]-1)):
            edge_list.append([coords_to_id(x_pos, y_pos), coords_to_id(x_pos+1, y_pos+1)])

edge_list = torch.tensor(edge_list).to(device).T

Construct dataloader object, now 100% ready for any pytorch geometric workflow

In [ ]:
x = torch.tensor(inputs).view(inputs.shape[0], -1, inputs.shape[-1]).to(device)
data_list = []
for i in range(100):
    data_list.append(pyg.data.Data(x=x[i], edge_index=edge_list, y=get_kinetic_engergy(inputs[i])))
loader = DataLoader(data_list, batch_size=32)

Sanity check: convert torch_geoemtric data object into NetworkX graph and visualize

In [ ]:
i=0
G = pyg.utils.to_networkx(data_list[i], to_undirected=True)
pos = {n:id_to_coords(n) for n in G.nodes}

plt.figure(figsize=(6, 6))
nx.draw(
    G, pos,
    node_size=15,
    node_color=[np.linalg.norm(a) for a in data_list[0].x.detach().cpu().numpy()],
    edge_color="#444444",
    width=2,
    font_size=8
)
plt.title(f'Graph {i} colored by total velocity. Total kinetic energy: {np.round(data_list[i].y, 2)}')
plt.show()

### Representation as cubical complex (topomodelx)
I still have to figure out how this works :/

In [ ]:
import toponetx as tnx
import topomodelx as tmx

In [ ]:
G.edges

In [ ]:
cubical_complex = tnx.transform.graph_to_clique_complex(G)

In [ ]:
cubical_complex.complex

# Questions
- Which type of energy (instantanious total kinetic energy)?
- How much smoothing?
- What are nodes and edges (evenly distributed? con)
